# **Testing the SIGA Model Without Fine-tuning**
## In this notebook, we will evaluate the performance of the pre-trained SIGA model (Microsoft DeBERTa) on our dataset.
## We will compare its performance with a fine-tuned version to assess the impact of fine-tuning on classification accuracy.

### **1. Install Required Libraries**
### Ensure that scikit-learn is installed before proceeding.

In [23]:
!pip install scikit-learn

import torch
import pandas as pd
import sklearn
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import classification_report, accuracy_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


### **2. Load the Dataset**
### Read the dataset containing premises, statements, and labels for Natural Language Inference (NLI).
### (remember to replace the path to your data file)

In [24]:
df = pd.read_csv('/home/wtto/Downloads/evaluation_dataset.csv')
display(df.tail())  # Display first few rows of the dataset

,Scale,premise,alternative,statement,label
1889,special,if you think you have given your best effort.....,tired,i'm still tired but i can't fall back asleep,neutral
1890,adequate,i'm using the next one up i think ? it's prett...,wary,warne wary of risking longevity at millers by ...,neutral
1891,intelligent,a woman that's intelligent is pretty damn attr...,special,the worst feeling is when someone makes you fe...,neutral
1892,allowed,senior portaits are tomorrow by last name. guy...,few,"and the fact people are able to say ""no."" is b...",neutral
1893,small,a very small sample of today's work out lol .....,some,some people make the mistake of thinking that ...,neutral


### **3. Load Pre-trained DeBERTa Model and Tokenizer**
### We use the `"microsoft/deberta-base-mnli"` model for sequence classification.

In [25]:
model_name = "microsoft/deberta-base-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()  # Set model to evaluation mode

/home/wtto/Documents/HHU/Third_Semester/NLI_AP/NLIvenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a Ber

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (

### **4. Move Model to GPU if Available**

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (

### **5. Define Batch Processing Parameters**

In [27]:
batch_size = 8  # Adjust based on available GPU memory
predicted_labels = []  # Store model predictions
true_labels = []  # Store ground truth labels

label_map = {
    "contradiction": 0,
    "neutral": 1,
    "entailment": 2
}


### **6. Process the Dataset in Batches**

In [28]:
# Process the dataset in batches
for i in range(0, len(df), batch_size):
    batch_texts = df["premise"].iloc[i:i+batch_size].tolist()
    batch_statements = df["statement"].iloc[i:i+batch_size].tolist()
    batch_true_labels = df["label"].iloc[i:i+batch_size].tolist()

    # Tokenize in smaller batches
    inputs = tokenizer(batch_texts, batch_statements, truncation=True, padding=True, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_classes = torch.argmax(logits, dim=-1).tolist()

    # Map model predictions to labels
    batch_predicted_labels = predicted_classes  # Keep them as integers


    # Append the batch predictions and true labels to the overall list
    predicted_labels.extend(batch_predicted_labels)
    true_labels.extend([label_map[lbl] for lbl in batch_true_labels])  # Ensure integer labels

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### **7. Evaluate Model Performance**
### Add predictions to the DataFrame

In [29]:
df["predicted_label"] = predicted_labels

# Print classification report
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, labels=[0,1,2], target_names=["contradiction", "neutral", "entailment"]))


Classification Report:
               precision    recall  f1-score   support

contradiction       0.01      0.00      0.00       441
      neutral       0.20      0.68      0.30       500
   entailment       0.00      0.00      0.00       953

     accuracy                           0.18      1894
    macro avg       0.07      0.23      0.10      1894
 weighted avg       0.05      0.18      0.08      1894

